<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Анализ" data-toc-modified-id="Анализ-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Анализ</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Тестирование" data-toc-modified-id="Тестирование-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span><strong>Вывод</strong></a></span>

#  Прогнозирование заказов такси

Компания «Чётенькое такси» собрала исторические данные о заказах такси в аэропортах. Чтобы привлекать больше водителей в период пиковой нагрузки, нужно спрогнозировать количество заказов такси на следующий час. Постройте модель для такого предсказания.

Значение метрики *RMSE* на тестовой выборке должно быть не больше 48.

Вам нужно:

1. Загрузить данные и выполнить их ресемплирование по одному часу.
2. Проанализировать данные.
3. Обучить разные модели с различными гиперпараметрами. Сделать тестовую выборку размером 10% от исходных данных.
4. Проверить данные на тестовой выборке и сделать выводы.


Данные лежат в файле `taxi.csv`. Количество заказов находится в столбце `num_orders` (от англ. *number of orders*, «число заказов»).

## Подготовка

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from statsmodels.tsa.seasonal import seasonal_decompose

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, TimeSeriesSplit
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

from catboost import CatBoostRegressor

In [ ]:
data = pd.read_csv('/datasets/taxi.csv')

In [ ]:
data.info()

In [ ]:
data.isna().sum()

In [ ]:
data.head()

In [ ]:
data.tail()

In [ ]:
# меняем тип данных Datetime с object на datetime64
# устанавливаем индекс таблицы равным столбцу Datetime
# сортируем по индексам
# проверяем временной ряд на монотонность
data = data.astype({'datetime':'datetime64'})
data = data.set_index('datetime')
data = data.sort_index(ascending = True)
print(data.index.is_monotonic)

In [ ]:
# ресемплируем по одному часу
data = data.resample('1H').sum()

## Анализ

In [ ]:
data_analis = data.copy()

In [ ]:
# строим тренды и сезонность
decomposed = seasonal_decompose(data_analis)
plt.figure(figsize=(8,10))
plt.subplot(311)
decomposed.trend.plot(ax=plt.gca())
plt.title('Trend')
plt.subplot(312)
decomposed.seasonal.plot(ax=plt.gca()) # < напишите код здесь >
plt.title('Seasonality')
# plt.subplot(313)
# decomposed.resid.plot(ax=plt.gca())# < напишите код здесь >
# plt.title('Residuals')
plt.tight_layout()

In [ ]:
# изучаем приближенно сезонность на днях
plt.figure(figsize=(6, 8))
plt.subplot(311)
decomposed.seasonal['2018-03-01':'2018-03-03'].plot(ax=plt.gca())
plt.title('Сезонность за 3 дня')
plt.tight_layout()

In [ ]:
# ресемплируем по одному дню
data_day = data.resample('1D').sum()

In [ ]:
# строим тренды и сезонность
decomposed = seasonal_decompose(data_day)
plt.figure(figsize=(10, 8))
plt.subplot(311)
decomposed.seasonal['2018-03-01':'2018-04-01'].plot(ax=plt.gca()) # < напишите код здесь >
plt.title('Сезонность за 1 месяц')
plt.tight_layout()

In [ ]:
# для прогнозирования ряды должны быть стационарными,
# Стохастический процесс стационарный, если его распределение со временем не меняется.
# сделаем верменной ряд стационарным,  через набор разностей между соседними элементами временного ряда
data_analis = data_analis - data_analis.shift()


data_analis['mean'] = data_analis['num_orders'].rolling(7).mean()
data_analis['std'] = data_analis['num_orders'].rolling(7).std()
data_analis.plot()

In [ ]:
data_analis.head()

Создадим признаки для горизонта прогнозирования в один день. Так как тренды и сезонность привязаны к конкретной дате, представим нужную инормацию как отдельные столбцы: согласно диаграмме трендов количество вызовов такси зависит от месяца, дня и часа. Создаем календарные признаки месяца, дня и часа. А также формируем "отстающие значения", это может определить функция x(t) будет расти или уменьшаться.

In [ ]:
def make_features(data, max_lag, rolling_mean_size):
    data['dayofweek'] = data.index.dayofweek
    data['hour'] = data.index.hour
    for lag in range(1, max_lag + 1):
        data['lag_{}'.format(lag)] = data['num_orders'].shift(lag)
    data['rolling_mean'] = data['num_orders'].shift().rolling(rolling_mean_size).mean()

In [ ]:
make_features(data, 8, 24)

In [ ]:
data.head()

## Обучение

In [ ]:
#создаем таблицу выходных значений
table_of_rmse=[]
table_of_model=[]

In [ ]:
data.info()

In [ ]:
# делим выборкуна train и test в размере 10%
train, test = train_test_split(data, shuffle=False, test_size=0.1)
train = train.dropna()

features_train = train.drop(['num_orders'],axis=1)
target_train = train['num_orders']
features_test = test.drop(['num_orders'],axis=1)
target_test = test['num_orders']

In [ ]:
RANDOM_STATE = 42

In [ ]:
print(train.index.min(), train.index.max())
print(test.index.min(), test.index.max())

**Линейная регрессия с регуляризацией**

In [ ]:
model_ridge = Ridge(random_state=RANDOM_STATE)

#словарь с гиперпараметрами и значениями, которые хотим перебрать
param_grid_ridge = {
    'alpha': np.arange(0, 0.21, 0.01),
}
tscv = TimeSeriesSplit(n_splits=3)
gs_ridge = GridSearchCV(
    model_ridge,
    cv = tscv,
    param_grid=param_grid_ridge,
    scoring='neg_root_mean_squared_error',
    n_jobs=-1
)

gs_ridge.fit(features_train, target_train)

# лучшее значение RMSE на кросс-валидации
print(f'best_score: {gs_ridge.best_score_* -1}')

# лучшие гиперпараметры
print(f'best_params: {gs_ridge.best_params_}')

In [ ]:
table_of_rmse.append(gs_ridge.best_score_* -1)
table_of_model.append('model_ridge')

**RandomForestRegressor**

In [ ]:
param = {
    'n_estimators':[20,30],
    'max_features':[0.3,1]
}
model_rfr = RandomForestRegressor()
gs_rfr = GridSearchCV(
    model_rfr,
    param,
    cv=tscv,
    scoring='neg_root_mean_squared_error',
    n_jobs=-1)

gs_rfr.fit(features_train, target_train)

# лучшее значение RMSE на кросс-валидации
print(f'best_score: {gs_rfr.best_score_* -1}')

# лучшие гиперпараметры
print(f'best_params: {gs_rfr.best_params_}')

In [ ]:
table_of_rmse.append(gs_rfr.best_score_* -1)
table_of_model.append(model_rfr)

**Catboost**

In [ ]:
parameter = {'learning_rate': [0.03,0.1],
             'depth': [4,6,8],
             'l2_leaf_reg': [1,2,3]
              }
model_cbr = CatBoostRegressor()

gs_cbr = GridSearchCV(
    estimator = model_cbr,
    param_grid = parameter,
    cv = tscv, n_jobs=-1,
    scoring='neg_root_mean_squared_error')

gs_cbr.fit(features_train, target_train)

In [ ]:
# лучшее значение RMSE на кросс-валидации
print(f'best_score: {gs_cbr.best_score_* -1}')

# лучшие гиперпараметры
print(f'best_params: {gs_cbr.best_params_}')

In [ ]:
table_of_rmse.append(gs_cbr.best_score_* -1)
table_of_model.append('model_cbr')

In [ ]:
models = (pd.DataFrame(
    {'Model':table_of_model,'RMSE':table_of_rmse})
                .sort_values(by='RMSE')
                .set_index('Model'))

models

**Вывод**  
По результатам обучения моделей лучшей оказалась Catboost c результатом метрики качества на train 23,95. Выбираем ее для тестирования.

## Тестирование

In [ ]:
pred = gs_cbr.predict(features_test)
RMSE_final_ = mean_squared_error(target_test, pred) ** 0.5
print('RMSE на тестовой выборке - ', RMSE_final_)

##### **Вывод**  
1. Загрузили данные и выполнили их ресемплирование по одному часу.
2. Проанализировали данные. Подготовили данные для обучения моделей.
3. Обучили три модели с различными гиперпараметрами. Сделали тестовую выборку размером 10% от исходных данных.
4. Провили данные на тестовой выборке.
5. По результатам обучения моделей лучшей оказалась Catboost c результатом метрики качества на train 26.7. выполнили предсказания на тестовой выборке. Метрика качества работы модели на тестовой выборке получилось 45, что соответствует заявленным требованиям к модели не больше 48.